In [9]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

df = pd.read_csv('data/ecommerce_customer_behavior_dataset.csv')

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Order_ID                  5000 non-null   object 
 1   Customer_ID               5000 non-null   object 
 2   Date                      5000 non-null   object 
 3   Age                       5000 non-null   int64  
 4   Gender                    5000 non-null   object 
 5   City                      5000 non-null   object 
 6   Product_Category          5000 non-null   object 
 7   Unit_Price                5000 non-null   float64
 8   Quantity                  5000 non-null   int64  
 9   Discount_Amount           5000 non-null   float64
 10  Total_Amount              5000 non-null   float64
 11  Payment_Method            5000 non-null   object 
 12  Device_Type               5000 non-null   object 
 13  Session_Duration_Minutes  5000 non-null   int64  
 14  Pages_Vi

,Order_ID,Customer_ID,Date,Age,Gender,City,Product_Category,Unit_Price,Quantity,Discount_Amount,Total_Amount,Payment_Method,Device_Type,Session_Duration_Minutes,Pages_Viewed,Is_Returning_Customer,Delivery_Time_Days,Customer_Rating
0,ORD_001337,CUST_01337,2023-01-01,27,Female,Bursa,Toys,54.28,1,0.00,54.28,Debit Card,Mobile,4,14,True,8,5
1,ORD_004885,CUST_04885,2023-01-01,42,Male,Konya,Toys,244.90,1,0.00,244.90,Credit Card,Mobile,11,3,True,3,3
2,ORD_004507,CUST_04507,2023-01-01,43,Female,Ankara,Food,48.15,5,0.00,240.75,Credit Card,Mobile,7,8,True,5,2
3,ORD_000645,CUST_00645,2023-01-01,32,Male,Istanbul,Electronics,804.06,1,229.28,574.78,Credit Card,Mobile,8,10,False,1,4
4,ORD_000690,CUST_00690,2023-01-01,40,Female,Istanbul,Sports,755.61,5,0.00,3778.05,Cash on Delivery,Desktop,21,10,True,7,4


In [11]:

initial_rows = len(df)

df['Date'] = pd.to_datetime(df['Date'])

df.drop_duplicates(inplace=True)
print(f"Rows removed due to duplication: {initial_rows - len(df)}")

#df.fillna(method='ffill', inplace=True) # Simple forward fill if any new nulls appeared

# 1. Normalize Categorical Text (Lowercase, Strip spaces, Title case for Tableau display)
categorical_cols = ['Product_Category', 'City', 'Gender', 'Payment_Method', 'Device_Type']
for col in categorical_cols:
    df[col] = df[col].astype(str).str.lower().str.strip().str.title()

# Use Capping (3 * IQR factor) to limit extreme values without deletion

def cap_outliers_iqr(df, col, factor=3):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + factor * IQR 
    lower_bound = Q1 - factor * IQR
    
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    return df

df = cap_outliers_iqr(df, 'Total_Amount')
df = cap_outliers_iqr(df, 'Delivery_Time_Days')
df = cap_outliers_iqr(df, 'Session_Duration_Minutes')


### 1. Temporal Features
df['Year'] = df['Date'].dt.year                         # Year (int)
df['Month_Name'] = df['Date'].dt.strftime('%B')         # English month names (string)
df['Day_Name'] = df['Date'].dt.day_name()               # English day names (string)
df['Is_Weekend'] = df['Date'].dt.weekday.isin([5, 6])   # True if is weekend (Bool)


### 2. Customer Segmentation

# a) Age Grouping
bins_age = [0, 25, 35, 45, 60, df['Age'].max() + 1] # Age Borders
labels_age = ['18-24 (Young)', '25-34 (Young Adult)', '35-44 (Middle Adult)', '45-59 (Senior)', '60+ (Retired)']
df['Age_Group'] = pd.cut(df['Age'], bins=bins_age, labels=labels_age, right=False)

# b) Spend Tier (Divided into Terciles)
df['Spend_Tier'] = pd.qcut(df['Total_Amount'], q=3, labels=['Low Value', 'Medium Value', 'High Value'])

### 3. Efficiency and Behavior Metrics

# a) Revenue Per Page View
df['Revenue_Per_Page'] = df['Total_Amount'] / df['Pages_Viewed']

# b) Value Per Minute
df['Value_Per_Minute'] = df['Total_Amount'] / df['Session_Duration_Minutes']

# Handle Inf/NaN values resulting from division by zero
cols_to_clean = ['Revenue_Per_Page', 'Value_Per_Minute']
for col in cols_to_clean:
    df[col].replace([np.inf, -np.inf], 0, inplace=True)
    df[col] = df[col].fillna(0)

# Save the final clean DataFrame with new features
df.to_csv('data/ecommerce_cleaned.csv', index=False, encoding='utf-8')

print("File 'data/ecommerce_cleaned.csv' is ready for Tableau connection.")

Rows removed due to duplication: 0
File 'data/ecommerce_cleaned.csv' is ready for Tableau connection.
